In [8]:
from tensorflow.keras.applications import InceptionV3

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [9]:
import os

In [10]:
IMAGE_SIZE = [224, 224]

In [11]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [12]:
!kaggle datasets download -d hereisburak/pins-face-recognition

 97% 362M/372M [00:02<00:00, 110MB/s]
100% 372M/372M [00:02<00:00, 136MB/s]


In [13]:
!unzip /content/pins-face-recognition.zip -d ./

Streaming output truncated to the last 5000 lines.
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner37_964.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner38_965.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner39_966.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner3_956.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner41_968.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner42_969.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner44_970.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner45_971.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner46_972.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner47_973.jpg  
  inflating: ./105_classes_pins_dataset/pins_Sophie Turner/Sophie Turner48_974.jpg  
  inflating: ./

In [14]:
import os
import cv2
import matplotlib.pyplot as plt

train_ratio = 0.8
test_ratio = 0.2
pre = r'/content/'
parent = '105_classes_pins_dataset'
dest = 'splitted_data'
# print(os.listdir(parent))

for folder in os.listdir(os.path.join(pre,parent)):
    if not os.path.exists(os.path.join(pre,dest,'train',folder)) and not os.path.exists(os.path.join(pre,dest,'test',folder)):
        os.makedirs(os.path.join(pre,dest,'train',folder))
        os.makedirs(os.path.join(pre,dest,'test',folder))
        
    images = os.listdir(os.path.join(pre,parent,folder))
    print(len(images))
    training_len = int(len(images)*0.8)
    for i,img in enumerate(images):
        timg = cv2.imread(os.path.join(pre,parent,folder,img))
        if i < training_len:
            cv2.imwrite(os.path.join(pre,dest,'train',folder,f"img{i}.png"),timg)
        else:
            cv2.imwrite(os.path.join(pre,dest,'test',folder,f"img{i}.png"),timg)
            

171
150
198
181
125
203
156
116
158
133
184
159
182
161
186
117
156
210
221
167
105
212
131
154
162
178
213
113
86
209
133
169
159
180
154
146
159
139
192
171
162
166
141
160
139
154
175
98
204
162
188
127
159
102
187
180
106
196
126
211
138
193
148
186
152
135
133
233
179
138
160
209
124
179
178
119
178
199
183
226
176
211
122
189
197
169
122
166
192
141
95
237
203
191
167
225
221
191
117
177
198
201
195
218
181


In [17]:
train_path = r"/content/splitted_data/train"
test_path = r"/content/splitted_data/test"

In [18]:
i_model = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [19]:
# don't train existing weights
for layer in i_model.layers:
    layer.trainable = False

In [20]:
# useful for getting number of output classes
folders = glob(r'/content/splitted_data/train/*')

In [21]:
# our layers - you can add more if you want
x = Flatten()(i_model.output)

In [22]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=i_model.input, outputs=prediction)

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [24]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [26]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r"/content/splitted_data/train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 13979 images belonging to 105 classes.


In [27]:
test_set = test_datagen.flow_from_directory(r"/content/splitted_data/test",
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3555 images belonging to 105 classes.


In [28]:
model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
437/437 [==============================] - 256s 552ms/step - loss: 17.5046 - accuracy: 0.1436 - val_loss: 15.9944 - val_accuracy: 0.2248
Epoch 2/20
437/437 [==============================] - 235s 539ms/step - loss: 13.8222 - accuracy: 0.2964 - val_loss: 16.7339 - val_accuracy: 0.2669
Epoch 3/20
437/437 [==============================] - 236s 540ms/step - loss: 11.9983 - accuracy: 0.3912 - val_loss: 17.5842 - val_accuracy: 0.2835
Epoch 4/20
437/437 [==============================] - 235s 539ms/step - loss: 10.5814 - accuracy: 0.4501 - val_loss: 17.1545 - val_accuracy: 0.3136
Epoch 5/20
437/437 [==============================] - 236s 539ms/step - loss: 9.9216 - accuracy: 0.4915 - val_loss: 18.1450 - val_accuracy: 0.3308
Epoch 6/20
437/437 [==============================] - 237s 541ms/step - loss: 8.8646 - accuracy: 0.5367 - val_loss: 19.0494 - val_accuracy: 0.3345
Epoch 7/20
437/437 [==============================] - 233s 533ms/step - loss: 8.4271 - accuracy: 0.5649 - val_loss

In [29]:
model.save("inception_new.h5")